In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q transformers bitsandbytes datasets peft accelerate

In [ ]:
!nvidia-smi

In [ ]:
import json
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, EarlyStoppingCallback

from typing import List, Dict, Optional
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
class LoRATrainer:
    def __init__(self, data_path: str, model_name: str):
        self.data_path = data_path
        self.model_name = model_name

        self.lora_config = None
        self.bnb_config = None
        self.model = None

        self.tokenizer = None
        self.train_tokenized_data = None
        self.validation_tokenized_data = None
        self.test_tokenized_data = None

        self.train_dataset = None
        self.validation_dataset = None
        self.test_dataset = None

        if self.model_name is None or not self.model_name.strip():
            raise ValueError("Model name cannot be none or empty")

    def set_lora_conf(self, lora_config: Optional[Dict] = None):
        if lora_config is None or not lora_config:
            lora_config = {
                "r":8,
                "lora_alpha":16,
                "target_modules":['q_proj', 'v_proj'],
                "lora_dropout":0.05,
                "bias":"none",
                "task_type": TaskType.CAUSAL_LM
            }

        self.lora_config = LoraConfig(**lora_config)
        return self.lora_config

    def set_bnb_config(self, bnb_config: Optional[Dict] = None):
        if not bnb_config or bnb_config is None:
            bnb_config = {
                "load_in_4bit":True,
                "bnb_4bit_quant_type":'nf4',
                "bnb_4bit_compute_dtype":torch.bfloat16
            }

        self.bnb_config = BitsAndBytesConfig(**bnb_config)
        return self.bnb_config

    def load_model(self):
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config = self.bnb_config,
            device_map = "auto",
            trust_remote_code = True
        )
        self.model = get_peft_model(self.model, self.lora_config)
        return self.model

    def preprocess_data(self, batch: Dict):
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            trust_remote_code=True)

        template = "### Instruction:\n{}### Response:\n{}"
        texts = [template.format(inst, resp) for inst, resp in zip(batch["instruction"], batch["response"])]

        tokens = self.tokenizer(
            texts,
            padding='max_length',
            max_length=256,
            truncation=True,
            return_tensors='pt'
        )

        tokens['labels'] =  tokens['input_ids'].clone()
        return tokens

    def set_tokenized_data(self, data_type: str = 'train'):
        if data_type == "train":
            self.train_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.train_dataset.column_names)
            return self.train_tokenized_data
        elif data_type == "validation":
            self.validation_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.validation_dataset.column_names)
            return self.validation_tokenized_data
        elif data_type == "test":
            self.test_tokenized_data = self.train_dataset.map(self.preprocess_data, batched=True, remove_columns=self.test_dataset.column_names)
            return self.test_tokenized_data

    def train(self, training_args: Optional[Dict]=None, adapted_model_path: str='./tinyllama-lora-ftuned-adapted'):
        if not training_args or training_args is None:
            training_args = {
                "output_dir":'./tinyllama-lora-ftuned-ml',
                'per_device_train_batch_size':4,
                'gradient_accumulation_steps':4,
                'learning_rate':1e-3,
                'num_train_epochs':50,
                'fp16':True,
                'report_to':'none',
                'logging_steps':20,
                'save_strategy':'epoch',
                'eval_strategy': 'epoch',
                'save_total_limit': 3,
                'load_best_model_at_end': True,
                'metric_for_best_model': 'eval_loss',
                'greater_is_better': False,
                'remove_unused_columns': False,
                'label_names': ['labels']
            }

        self.training_args = TrainingArguments(**training_args)

        early_stopping = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01
        )

        trainer = Trainer(
            model = self.model,
            args = self.training_args,
            train_dataset = self.train_tokenized_data,
            eval_dataset = self.validation_tokenized_data,
            processing_class = self.tokenizer,
            callbacks=[early_stopping]
        )

        print("Training the model")
        trainer.train()

        print("Saving the new weights")
        self.model.save_pretrained(adapted_model_path)
        self.tokenizer.save_pretrained(adapted_model_path)

    def prepare_and_load_data(self):
            preprocessed_data = []
            data = None

            try:
                with open(self.data_path) as f:
                    data = json.load(f)
            except FileNotFoundError:
                print(f"Error: File not found -> {self.data_path}")
                return []
            except json.JSONDecodeError as e:
                print(f"Error: Invalid JSON format in {self.data_path} - {e}")
            except Exception as e:
                print(f"Unexpected error while loading file: {e}")
                return []

            questions = data.get("questions", [])
            if not isinstance(questions, list):
                print("Error: 'questions' should be a list in the dataset")
                return []

            for idx, q in enumerate(questions, start=1):
                try:
                    preprocessed_data.append({
                        "instruction": f"[{q['category']} - {q['difficulty']}] {q['instruction']}",
                        "input": "",
                        "response": q["response"]
                    })
                except KeyError as e:
                    print(f"Skipping question {idx}: Missing key {e}")
                except Exception as e:
                    print(f"Skipping question {idx}: due to unexpected error - {e}")

            with open("lora_dataset.json", "w") as f:
                json.dump(preprocessed_data, f, indent=2)

            dataset = load_dataset("json", data_files="lora_dataset.json")


            dataset = dataset['train'].train_test_split(test_size=0.3, seed=42)
            train_dataset = dataset['train']
            temp_dataset = dataset['test']

            temp_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
            validation_dataset = temp_split['train']  # 15% of total
            test_dataset = temp_split['test']         # 15% of total

            self.train_dataset = train_dataset        # 70%
            self.validation_dataset = validation_dataset  # 15%
            self.test_dataset = test_dataset          # 15%

            print(f"Train samples: {len(self.train_dataset)}")
            print(f"Validation samples: {len(self.validation_dataset)}")
            print(f"Test samples: {len(self.test_dataset)}")

            return self.train_dataset, self.validation_dataset, self.test_dataset

In [ ]:
data_path = "/content/drive/MyDrive/interview-questions/ml_interview_dataset.json"
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

In [ ]:
lora_trainer_obj = LoRATrainer(data_path=data_path, model_name=model_name)

In [ ]:
lora_trainer_obj.prepare_and_load_data()

In [ ]:
lora_trainer_obj.set_lora_conf()

In [ ]:
lora_trainer_obj.set_bnb_config()

In [ ]:
lora_trainer_obj.load_model()

In [ ]:
lora_trainer_obj.set_tokenized_data("train")
lora_trainer_obj.set_tokenized_data("validation")

In [ ]:
lora_trainer_obj.train()